In [2]:
! pip install -q kaggle

import os
from google.colab import files

if not os.path.exists("/root/.kaggle/kaggle.json"):
  files.upload()
  ! mkdir -p ~/.kaggle
  ! mv kaggle.json ~/.kaggle/ 
  ! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:
!kaggle datasets download fullmetal26/glovetwitter27b100dtxt 
!unzip glovetwitter27b100dtxt.zip

 99% 770M/778M [00:14<00:00, 54.3MB/s]
100% 778M/778M [00:14<00:00, 57.8MB/s]
Archive:  glovetwitter27b100dtxt.zip
  inflating: glove.twitter.27B.200d.txt  


In [19]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, SimpleRNN, Activation, Dropout, Conv1D
from tensorflow.keras.layers import Embedding, Flatten, LSTM, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import spacy
from sklearn.metrics import classification_report

In [5]:
def load_glove_model(glove_file):
    print("[INFO]Loading GloVe Model...")
    model = {}
    with open(glove_file, 'r') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embeddings = [float(val) for val in split_line[1:]]
            model[word] = embeddings
    print("[INFO] Done...{} words loaded!".format(len(model)))
    return model
glove_model = load_glove_model("glove.twitter.27B.200d.txt")

[INFO]Loading GloVe Model...
[INFO] Done...1193514 words loaded!


In [6]:
# adopted from utils.py
nlp = spacy.load("en")

def remove_stopwords(sentence):
    '''
    function to remove stopwords
        input: sentence - string of sentence
    '''
    new = []
    # tokenize sentence
    sentence = nlp(sentence)
    for tk in sentence:
        if (tk.is_stop == False) & (tk.pos_ !="PUNCT"):
            new.append(tk.string.strip())
    # convert back to sentence string
    c = " ".join(str(x) for x in new)
    return c


def lemmatize(sentence):
    '''
    function to do lemmatization
        input: sentence - string of sentence
    '''
    sentence = nlp(sentence)
    s = ""
    for w in sentence:
        s +=" "+w.lemma_
    return nlp(s)

def sent_vectorizer(sent, model):
    '''
    sentence vectorizer using the pretrained glove model
    '''
    sent_vector = np.zeros(200)
    num_w = 0
    for w in sent.split():
        try:
            # add up all token vectors to a sent_vector
            sent_vector = np.add(sent_vector, model[str(w)])
            num_w += 1
        except:
            pass
    return sent_vector

In [7]:
!wget https://raw.githubusercontent.com/haochen23/nlp-rnn-lstm-sentiment/master/training.1600000.processed.noemoticon.csv

--2020-06-19 10:20:33--  https://raw.githubusercontent.com/haochen23/nlp-rnn-lstm-sentiment/master/training.1600000.processed.noemoticon.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2989873 (2.9M) [text/plain]
Saving to: ‘training.1600000.processed.noemoticon.csv’

training.1600000.pr 100%[===================>]   2.85M  --.-KB/s    in 0.1s    

2020-06-19 10:20:34 (20.5 MB/s) - ‘training.1600000.processed.noemoticon.csv’ saved [2989873/2989873]



In [8]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", header=None, encoding='latin-1')
print(data.shape)
data.head()

(20000, 6)


,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
data_X = data[data.columns[5]]
data_y = data[data.columns[0]]
data_y = pd.get_dummies(data_y)

In [10]:
data_y = data_y.to_numpy()
data_y

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]], dtype=uint8)

In [11]:
max_vocab = 18000
max_len = 15
tokenizer = Tokenizer(num_words=max_vocab)

In [12]:
data_X = data_X.to_numpy()
print(data_X)
tokenizer.fit_on_texts(data_X)
sequences = tokenizer.texts_to_sequences(data_X)
word_index = tokenizer.word_index


["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
 "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"
 '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'
 ... 'Are you ready for your MoJo Makeover? Ask me for details '
 'Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur '
 'happy #charitytuesday @theNSPCC @SparksCharity @SpeakingUpH4H ']


In [13]:
print('Found %s unique tokens.' % len(word_index))

Found 30256 unique tokens.


In [14]:
data_keras = pad_sequences(sequences, maxlen=max_len, padding="post")
# data_keras = data_keras.reshape(data_X.shape[0], 15, 1)
data_keras.shape



(20000, 15)

In [15]:
from sklearn.model_selection import train_test_split
train_X, valid_X, train_y, valid_y = train_test_split(data_keras, data_y, test_size = 0.3, random_state=42)

In [16]:
# calcultaete number of words
nb_words = len(tokenizer.word_index) + 1

# obtain the word embedding matrix
embedding_matrix = np.zeros((nb_words, 200))
for word, i in word_index.items():
    embedding_vector = glove_model.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 12567


In [22]:
# adopted from sent_tran_eval.py
def build_model(nb_words, rnn_model="SimpleRNN", embedding_matrix=None):
    '''
    build_model function:
    inputs: 
        rnn_model - which type of RNN layer to use, choose in (SimpleRNN, LSTM, GRU)
        embedding_matrix - whether to use pretrained embeddings or not
    '''
    model = Sequential()
    # add an embedding layer
    if embedding_matrix is not None:
        model.add(Embedding(nb_words, 
                        200, 
                        weights=[embedding_matrix], 
                        input_length= max_len,
                        trainable = False))
    else:
        model.add(Embedding(nb_words, 
                        200, 
                        input_length= max_len,
                        trainable = False))
        
    # add an RNN layer according to rnn_model
    if rnn_model == "SimpleRNN":
        model.add(SimpleRNN(200))
    elif rnn_model == "LSTM":
        model.add(LSTM(200))
    else:
        model.add(GRU(200))
    # model.add(Dense(500,activation='relu'))
    # model.add(Dense(500, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', 
                optimizer='adam',
                metrics=['accuracy'])
    return model

In [23]:
model_rnn = build_model(nb_words, "SimpleRNN", embedding_matrix)
model_rnn.fit(train_X, train_y, epochs=20, batch_size=120,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))
predictions = model_rnn.predict(valid_X)
predictions = predictions.argmax(axis=1)
print(classification_report(valid_y.argmax(axis=1), predictions))

Epoch 1/20
117/117 [==============================] - 4s 38ms/step - loss: 0.5867 - accuracy: 0.6791 - val_loss: 0.5106 - val_accuracy: 0.7390
Epoch 2/20
117/117 [==============================] - 4s 36ms/step - loss: 0.4854 - accuracy: 0.7678 - val_loss: 0.5161 - val_accuracy: 0.7440
Epoch 3/20
117/117 [==============================] - 4s 32ms/step - loss: 0.4419 - accuracy: 0.7939 - val_loss: 0.5366 - val_accuracy: 0.7468
Epoch 4/20
117/117 [==============================] - 4s 34ms/step - loss: 0.4068 - accuracy: 0.8162 - val_loss: 0.5075 - val_accuracy: 0.7685
Epoch 5/20
117/117 [==============================] - 4s 32ms/step - loss: 0.3717 - accuracy: 0.8358 - val_loss: 0.5024 - val_accuracy: 0.7648
Epoch 6/20
117/117 [==============================] - 4s 35ms/step - loss: 0.3343 - accuracy: 0.8541 - val_loss: 0.5503 - val_accuracy: 0.7512
Epoch 7/20
117/117 [==============================] - 4s 33ms/step - loss: 0.2923 - accuracy: 0.8791 - val_loss: 0.5762 - val_accuracy: 0.7637

In [24]:
model_lstm = build_model(nb_words, "LSTM", embedding_matrix)
model_lstm.fit(train_X, train_y, epochs=20, batch_size=120,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))
predictions = model_lstm.predict(valid_X)
predictions = predictions.argmax(axis=1)
print(classification_report(valid_y.argmax(axis=1), predictions))


Epoch 1/20
117/117 [==============================] - 11s 96ms/step - loss: 0.5433 - accuracy: 0.7152 - val_loss: 0.4865 - val_accuracy: 0.7682
Epoch 2/20
117/117 [==============================] - 11s 93ms/step - loss: 0.4775 - accuracy: 0.7679 - val_loss: 0.4858 - val_accuracy: 0.7667
Epoch 3/20
117/117 [==============================] - 11s 93ms/step - loss: 0.4545 - accuracy: 0.7835 - val_loss: 0.4625 - val_accuracy: 0.7797
Epoch 4/20
117/117 [==============================] - 11s 94ms/step - loss: 0.4278 - accuracy: 0.8028 - val_loss: 0.4629 - val_accuracy: 0.7750
Epoch 5/20
117/117 [==============================] - 11s 93ms/step - loss: 0.4011 - accuracy: 0.8149 - val_loss: 0.4584 - val_accuracy: 0.7858
Epoch 6/20
117/117 [==============================] - 11s 94ms/step - loss: 0.3681 - accuracy: 0.8336 - val_loss: 0.4475 - val_accuracy: 0.7867
Epoch 7/20
117/117 [==============================] - 11s 96ms/step - loss: 0.3320 - accuracy: 0.8546 - val_loss: 0.4769 - val_accuracy:

In [25]:
model_gru = build_model(nb_words, "GRU", embedding_matrix)
model_gru.fit(train_X, train_y, epochs=20, batch_size=120,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))
predictions = model_gru.predict(valid_X)
predictions = predictions.argmax(axis=1)
print(classification_report(valid_y.argmax(axis=1), predictions))

Epoch 1/20
117/117 [==============================] - 10s 82ms/step - loss: 0.5690 - accuracy: 0.6926 - val_loss: 0.4837 - val_accuracy: 0.7698
Epoch 2/20
117/117 [==============================] - 9s 79ms/step - loss: 0.4757 - accuracy: 0.7695 - val_loss: 0.4635 - val_accuracy: 0.7838
Epoch 3/20
117/117 [==============================] - 9s 79ms/step - loss: 0.4435 - accuracy: 0.7888 - val_loss: 0.4606 - val_accuracy: 0.7832
Epoch 4/20
117/117 [==============================] - 9s 79ms/step - loss: 0.4178 - accuracy: 0.8041 - val_loss: 0.4696 - val_accuracy: 0.7742
Epoch 5/20
117/117 [==============================] - 9s 78ms/step - loss: 0.3847 - accuracy: 0.8267 - val_loss: 0.4581 - val_accuracy: 0.7848
Epoch 6/20
117/117 [==============================] - 9s 79ms/step - loss: 0.3462 - accuracy: 0.8485 - val_loss: 0.4884 - val_accuracy: 0.7735
Epoch 7/20
117/117 [==============================] - 9s 78ms/step - loss: 0.3062 - accuracy: 0.8688 - val_loss: 0.4865 - val_accuracy: 0.783